In [1]:
import time
start = time.time()

In [2]:
#IMPORTS
from datetime import datetime
from pyspark.ml.fpm import PrefixSpan
from pyspark.sql import Row
import pyspark.sql.functions as F
from math import sin, cos, sqrt, atan2, radians 
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt
import operator
import copy


Bad key "text.kerning_factor" on line 4 in
/opt/anaconda3/envs/bigdatalab_cpu_202101/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


In [3]:
#parameters
interval=30 #time window
th=1 #distance
window_size=3 #window size
maxDelta=3 #how many delta
support=0
support_str="0"
station_status= "StateChange_Normal_almostFull"

In [162]:
#FILES
#Considero soltanto le stazioni di San Francisco
inputPath  = "file:///home/bigdata-01QYD/s270240/bike_sharing/Associative_classifier/Datasets/SanFrancisco_status_train.csv"
STATION_PATH="file:///home/bigdata-01QYD/s270240/bike_sharing/station.csv"
#save file of first filter
output_file=f'./Results_extraction/{interval}min_{int(th*1000)}m/{results_{station_status}_{interval}_{int(th*1000)}_{support_str}({window_size}-{maxDelta})_ordered_by_confidence.txt'
output_file_conf1=f'./Results_extraction/{interval}min_{int(th*1000)}m/Time_slots/results_6-10_{station_status}_{interval}_{int(th*1000)}_{support_str}({window_size}-{maxDelta})_ordered_by_confidence.txt'
output_file_conf2=f'./Results_extraction/{interval}min_{int(th*1000)}m/Time_slots/results_10-14_{station_status}_{interval}_{int(th*1000)}_{support_str}({window_size}-{maxDelta})_ordered_by_confidence.txt'
output_file_conf3=f'./Results_extraction/{interval}min_{int(th*1000)}m/Time_slots/results_14-17_{station_status}_{interval}_{int(th*1000)}_{support_str}({window_size}-{maxDelta})_ordered_by_confidence.txt'
output_file_conf4=f'./Results_extraction/{interval}min_{int(th*1000)}m/Time_slots/results_17-20_{station_status}_{interval}_{int(th*1000)}_{support_str}({window_size}-{maxDelta})_ordered_by_confidence.txt'
output_file_conf5=f'./Results_extraction/{interval}min_{int(th*1000)}m/Time_slots/results_20-24_{station_status}_{interval}_{int(th*1000)}_{support_str}({window_size}-{maxDelta})_ordered_by_confidence.txt'
output_file_conf6=f'./Results_extraction/{interval}min_{int(th*1000)}m/Time_slots/results_0-6_{station_status}_{interval}_{int(th*1000)}_{support_str}({window_size}-{maxDelta})_ordered_by_confidence.txt'



In [5]:
inputDF = spark.read.format("csv")\
.option("delimiter", ",")\
.option("header", True)\
.option("inferSchema", True).load(inputPath)

In [6]:
inputDF=inputDF.filter("docks_available is not null")

In [7]:
#filter for these fields
filteredDF = inputDF.filter("docks_available<>0 OR bikes_available<>0")

In [8]:
# SFancisco_stations = [41, 42, 45, 46, 47, 48, 49, 50, 51, 39, 54, 55, 56, 57, 58, 59, 60, 
#                       61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 82]
# filteredDF = filteredDF[filteredDF['station_id'].isin(SFancisco_stations)]
# filteredDF.collect()

In [9]:
#determine if the station is normal or almost full
def stateFunction(docks_available,bikes_available):
    if docks_available<=2: # almostFull
        return 1
    else:         #normal
        return 0

In [10]:
spark.udf.register("state", stateFunction)

<function __main__.stateFunction(docks_available, bikes_available)>

In [11]:
def getInt(station):
    return (station)

In [12]:
spark.udf.register("intValue", getInt)

<function __main__.getInt(station)>

In [13]:
getStatusDF = filteredDF.selectExpr("station_id","time", "state(docks_available,bikes_available) as status")

In [14]:
# getStatusDF.show(100,False)

In [15]:
#create a view
getStatusDF.createOrReplaceTempView("readings")

In [16]:
#select station, year, month, day, hour, minute, status ordered by time
ss=spark.sql("""SELECT  station_id , YEAR(time) as year, MONTH(time) as month, DAY(time) as day, HOUR(time)as hour, MINUTE(time) as minute, status
FROM readings
GROUP BY station_id, year, month, day,hour,minute, status
ORDER BY  station_id,year, month,day, hour,minute
""")

In [17]:
#create rdd and group into interval
my_rdd=ss.rdd.map(tuple)
rdd=my_rdd.map(lambda line: (line[0],line[1],line[2], line[3], line[4], int(line[5]/interval), line[6])).distinct()

In [18]:
# rdd.collect()

# [(6, 2013, 12, 31, 1, 1, '2'),
#  (6, 2014, 1, 14, 22, 1, '2'),
#  (6, 2014, 1, 16, 0, 1, '2'),
#  (6, 2014, 1, 19, 6, 0, '2'),
#  (6, 2014, 1, 23, 21, 1, '2'),
#  (6, 2014, 1, 26, 10, 1, '2'),
#  (6, 2014, 1, 30, 14, 1, '2'), ... ]

In [19]:
#get distinct stations to calculate distances
id_stations=rdd.map(lambda line: line[0]).distinct()

In [20]:
tot_id_stations=id_stations.collect()

In [21]:
#obtain timestamp and info
def getMap2(line):
    id_station=str(line[0])
    year=int(line[1])
    month=int(line[2])
    day=int(line[3])
    hour=int(line[4])
    minute=int(line[5])   
    timestamp= datetime(year,month, day, hour, minute)  
    status=int(line[6])
    info=str(id_station)+' '+str(status)
    return (timestamp, info)

In [22]:
get_map2=rdd.map(getMap2)

In [23]:
# get_map2.collect()

# [(datetime.datetime(2014, 3, 31, 18, 1), '11 2'),
#  (datetime.datetime(2014, 4, 6, 21, 0), '11 2'),
#  (datetime.datetime(2014, 4, 10, 22, 1), '11 2'),
#  (datetime.datetime(2014, 4, 12, 8, 1), '11 2'),
#  (datetime.datetime(2014, 4, 13, 8, 0), '11 2'), ... ]

In [24]:
reduceK2=get_map2.reduceByKey(lambda l1,l2 :(l1+','+l2)).sortByKey()

In [25]:
# reduceK2.collect()

# [(datetime.datetime(2013, 8, 29, 10, 0), '59 0,67 0,4 1,13 1,67 1,6 0,2 0,7 1,16 0,70 1,70 0'),
#  (datetime.datetime(2013, 8, 29, 10, 1), '67 0,7 1,16 0,59 0,13 1,67 1,70 0,2 0,7 0,69 0,4 1,13 0,73 0'),
#  (datetime.datetime(2013, 8, 29, 11, 0), '67 1,2 0,4 0,73 0,67 0,69 0,16 0,70 0,4 1'),
#  (datetime.datetime(2013, 8, 29, 11, 1), '16 0,73 0,2 0,69 0,67 0'),
#  (datetime.datetime(2013, 8, 29, 12, 0), '69 1,69 0,16 0,2 0,73 0'), ... ]

In [26]:
#divide rdd in different time slots
slot1= reduceK2.filter(lambda line : line[0].hour >=6 and line[0].hour<10)
slot2= reduceK2.filter(lambda line : line[0].hour >=10 and line[0].hour<14)
slot3= reduceK2.filter(lambda line : line[0].hour >=14 and line[0].hour<17)
slot4= reduceK2.filter(lambda line : line[0].hour >=17 and line[0].hour<20)
slot5= reduceK2.filter(lambda line : line[0].hour >=20 and line[0].hour<=23)
slot6= reduceK2.filter(lambda line : line[0].hour >=0 and line[0].hour<6)

#create a list contaning all slots rdd
slots = [slot1, slot2, slot3, slot4, slot5, slot6]

In [28]:
list_tots = []
for i, slot in enumerate(slots):
    list_tots.append(slot.collect())

In [85]:
print(len(list_tots))
# list_tot[:5]

6


In [31]:
multiple_list_filtered = []

for list_tot in list_tots:

    voc_status={}
    voc_timestamp={}
    list_filtered=[]

    voc_tmp={}

    # previous_list=[]
    # updated_list=[]

    for i, el in enumerate(list_tot):
        #updated_list=[]    
        information=el[1]   
        tmp_list_station='' 
        voc_tmp={}

        for info in information.split(','):
    #         print()
    #         print(info)
            station=str(info.split(' ')[0])

            status=int(info.split(' ')[1])
            if  station not in voc_status:
    #             print (f"station: {station} NOT PRESENT")
                voc_status[station]=status
                tmp_time= el[0].replace(minute=el[0].minute*interval) #riconverto nei minuti originali 
                voc_timestamp[station]=tmp_time
    #             print (f"actual_time: {voc_timestamp[station]}")
                stringa=''
                if status==1:
                    stringa='QuasiPiena'
                else:
                    stringa='Normal'    
                tmp_list_station+=','+station+'_'+ stringa 
    #             print(tmp_list_station)
                #updated_list.append(station)

            else:
    #             print (f"station: {station} PRESENT")
                times=voc_timestamp[station]
    #             print (f"times: {times}")
                tmp_time= el[0].replace(minute=el[0].minute*interval) #riconverto nei minuti originali
                actual_time=tmp_time
    #             print (f"actual_time: {actual_time}")
    #             print (f"old satus: {voc_status[station]}")
    #             print (f"new satus: {status}")

                if times == actual_time:    #inside same timeslot
                    if status != voc_status[station]:   # if status changed   
    #                     print("STATUS CHANGED INSIDE THE SAME TIMESLOT!!!!!")
                        voc_status[station]=3  #set flag of status changed inside same timeslot
                        stringa=''
                        if status==1:
                            stringa='QuasiPiena'
                        else:
                            stringa='Normal'    
                        tmp_list_station+=','+station+'_'+ stringa 
    #                     print(tmp_list_station)

                else:            # different timeslot
    #                 print(f"actual_time.minute-times.minute: {int((actual_time-times).total_seconds() / 60.0)}")
    #                 if ((actual_time.hour == times.hour and actual_time.minute-times.minute==1) or
    #                     (actual_time.hour-times.hour==1 and actual_time.minute-times.minute==-1)):

                    if (int((actual_time-times).total_seconds() / 60.0)== interval): # after less than 30 minutes
    #                     print(f"after <= 30 minutes")
                        if voc_status[station]==3:         #if status changed inside the same old timeslot        
    #                         print("STATUS CHANGED INSIDE THE SAME OLD TIMESLOT!!!!!")
                            voc_status[station]=status     #update old status with new one
                            voc_timestamp[station]=actual_time      #update old timestamp with new one
                            stringa=''
                            if status==1:
                                stringa='QuasiPiena'
                            else:
                                stringa='Normal'
                            tmp_list_station+=','+station+'_'+ stringa 
    #                         print(tmp_list_station)
                            #updated_list.append(station)

                        elif voc_status[station] != status:  # if status changed between old and new timeslot 
    #                         print("STATUS CHANGED!!!!!")
                            voc_status[station]=status     #update old status with new one
                            voc_timestamp[station]=actual_time    #update old timestamp with new one
                            stringa=''
                            if status==1:
                                stringa='QuasiPiena'
                            else :
                                stringa='Normal'
                            tmp_list_station+=','+station+'_'+ stringa 
    #                         print(tmp_list_station)

                            if station in voc_tmp:
                                if voc_tmp[station]==1:
                                     tmp_list_station+=','+station+'_QuasiPiena'
                                elif voc_tmp[station]==0:
                                     tmp_list_station+=','+station+'_Normal'

    #                             print(tmp_list_station)


                        elif  voc_status[station] == status: # if status did not change between old and new timeslot
    #                         print("STATUS DID NOT CHANGE!!!!!")
                            voc_tmp[station]=status
                            tmp_list_station+=','+station+'_NoChange'
    #                         print(tmp_list_station)
    #                         voc_timestamp[station]=actual_time     #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

                    # after more than 30 minutes        
                    else:
    #                     print(f"after >= 30 minutes")
                        if voc_status[station] != status:
                            stringa=''
                            if status==1:
                                stringa='QuasiPiena'
                            else :
                                stringa='Normal'
                            tmp_list_station+=','+station+'_'+ stringa 
    #                         print(tmp_list_station)
                        else:
                            tmp_list_station+=','+station+'_NoChange'
    #                     print (f"REMOVING STATION {station}")    
    #                     del voc_status[station]
    #                     del voc_timestamp[station]
                        voc_timestamp[station]=actual_time


        list_filtered.append((el[0],tmp_list_station[1::]))
    
    multiple_list_filtered.append(list_filtered)

In [87]:
multiple_reduceK = []

for list_filtered in multiple_list_filtered:
    multiple_reduceK.append(spark.sparkContext.parallelize(list_filtered))


In [90]:
print(len(multiple_reduceK))
# reduceK.collect()

6


In [91]:
my_df_list = []

for reduceK in multiple_reduceK:
    my_df_list.append(reduceK.toDF())
#my_df.collect()

In [92]:
for i, my_df in enumerate(my_df_list):
    my_df.createOrReplaceTempView(f"view{i}")

In [93]:
add_rows_list = []
for i in range(0,len(my_df_list)):
    s2 = spark.sql(f"""SELECT ROW_NUMBER() OVER(ORDER BY _1,_2) as id ,_1, _2
                FROM view{i} """)
    add_rows_list.append(s2)


# s2=spark.sql("""SELECT ROW_NUMBER() OVER(ORDER BY _1,_2) as id ,_1, _2
# FROM view """)

In [94]:
#identifier of the timestamp, info
rdd_scheme_list = []
for s2 in add_rows_list:
    rdd_scheme=s2.rdd.map(tuple).map(lambda line: (line[0], line[2]))
    rdd_scheme_list.append(rdd_scheme)


# rdd_scheme=s2.rdd.map(tuple).map(lambda line: (line[0], line[2]))

In [95]:
# rdd_scheme.collect()

In [96]:
#obtain window, station-status
def giveSplit(line):   
    id_window=( int(line[0] ))
    lista=[]    
    counter=id_window    
    while counter>=1:
        lista.append(('Window '+str(counter),(line[1])))
        counter=counter-1
        if (id_window-counter)==window_size:
            return lista  
    return lista

In [97]:
mapData_list = []

for rdd_scheme in rdd_scheme_list:
    mapData=rdd_scheme.flatMap(giveSplit)
    mapData_list.append(mapData)

# mapData=rdd_scheme.flatMap(giveSplit)

In [98]:
# mapData.collect()

In [99]:
filteredD_list = []
for mapData in mapData_list:
    filteredD=mapData.filter(lambda line: line[1]!='')
    filteredD_list.append(filteredD)


In [100]:
# filteredD.collect()

In [101]:
# for each window get all info
all_keys_list = []

for filteredD in filteredD_list:
    all_keys=filteredD.reduceByKey(lambda l1,l2:(l1+'-'+l2))    
    all_keys_list.append(all_keys)


# all_keys=filteredD.reduceByKey(lambda l1,l2:(l1+'-'+l2))

In [102]:
# all_keys.collect()

In [103]:
#finestra temporale
def reduceKeys(line):   
    lista=[]
    #lista.append(line[0])
    line_split=line[1].split("-")
    #return line_split[0]
    count=len(line_split)
    tot=[]
    for val in range(count):
        li=[]
        stations=line_split[val].split(',')
        for st in stations:
            all_string_st=st.split('_')[0]+'_'+'T'+str(val)+'_'+st.split('_')[1]
            li.append(all_string_st)
        tot.append(li)
    lista.append((line[0],(tot))) 
    return lista

In [104]:
windows_list = []

for all_keys in all_keys_list:
    windows=all_keys.flatMap(reduceKeys)
    windows_list.append(windows)

# windows=all_keys.flatMap(reduceKeys)

In [ ]:
# windows_list[5].collect()

In [109]:
#filtering NoChange events
def filterNormal(line):
    complete_list = []
    for list_el in line[1]:
        event_list = copy.deepcopy(list_el)
        for el in list_el:
            if "NoChange"in el:
                event_list.remove(el)
#         new_event_list = ",".join(event_list)
        complete_list.append(event_list)
    return (line[0], complete_list)

In [110]:
#remove normal and noChange events
windows_withoutNormal_list = []

for windows in windows_list:
    windows_withoutNormal = windows.map(filterNormal)
    windows_withoutNormal_list.append(windows_withoutNormal)
    
# windows_withoutNormal = windows.map(filterNormal)


In [111]:
# windows_withoutNormal.collect()

# [('Window 1',
#   [[],
#    ['60_T1_QuasiPiena', '64_T1_QuasiPiena', '45_T1_Piena', '45_T1_QuasiPiena'],
#    ['45_T2_Piena', '64_T2_Piena', '64_T2_QuasiPiena', '45_T2_QuasiPiena']]),
#  ('Window 2',
#   [['60_T0_QuasiPiena', '64_T0_QuasiPiena', '45_T0_Piena', '45_T0_QuasiPiena'],
#    ['45_T1_Piena', '64_T1_Piena', '64_T1_QuasiPiena', '45_T1_QuasiPiena'],
#    ['64_T2_QuasiPiena', '35_T2_QuasiPiena']]),... ]

In [112]:
#filtering empty list ''
def filterEmptyList(line):
    filtered_list = list(filter(None, line[1]))
    return (line[0], filtered_list)

In [113]:
filtered_windows_list =[]

for windows_withoutNormal in windows_withoutNormal_list:
    filtered_windows = windows_withoutNormal.map(filterEmptyList)
    filtered_windows_list.append(filtered_windows)


# filtered_windows = windows_withoutNormal.map(filterEmptyList)
# filtered_windows.collect()

In [114]:
#filtering empty window
filtered_windows2_list = []

for filtered_windows in filtered_windows_list:
    filtered_windows2 = filtered_windows.filter(lambda line: line[1])
    filtered_windows2_list.append(filtered_windows2)


# filtered_windows2 = filtered_windows.filter(lambda line: line[1])
# filtered_windows2.collect()

In [115]:
#save station file
stationsDF = spark.read.format("csv")\
.option("delimiter", ",")\
.option("header", True)\
.option("inferSchema", True).load(STATION_PATH)

In [116]:
#get only rows interested: only the used stations 
necessary_rows=stationsDF.filter(F.col("id").isin(tot_id_stations)).sort("id").rdd.map(tuple)

In [117]:
#necessary_rows.collect()

In [118]:
#get info of stations about coordinates and name
coordinates=necessary_rows.map(lambda line: (line[0],(str(line[2])+','+str(line[3]))))
names_stations=necessary_rows.map(lambda line: (line[0],line[1]))

In [119]:
list_coo=coordinates.collect()

In [120]:
# list_coo

# [(2, '37.329732,-121.90178200000001'),
#  (3, '37.330698,-121.888979'),
#  (4, '37.333988,-121.894902'),
#  (5, '37.331415,-121.8932'),
#  (6, '37.336721000000004,-121.894074'), ...]

In [121]:
#create a dictionary in which the key is the station and value is the info about coordinates
dic_co=coordinates.collectAsMap()
dic_coordinates=sc.broadcast(dic_co)
#dic_coordinates.value[1]

In [122]:
#function to retrieve distance between 2 stations
def getDistance(station1,station2):
    # approximate radius of earth in km
    R = 6373.0    
    lat_a=float(station1.split(',')[0])
    lat_b=float(station2.split(',')[0])
    long_a=float(station1.split(',')[1])
    long_b=float(station2.split(',')[1])
    
    lat1=radians(lat_a)
    lat2=radians(lat_b)
    lon1=radians(long_a)
    lon2=radians(long_b)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [123]:
#voc in which the key is a pair of stations and value is the distance
voc_distances={}
for i in range(len(list_coo)):
    for j in range(i+1,len(list_coo)):
        station1=list_coo[i][0]
        station2=list_coo[j][0]
        d_i=list_coo[i][1]
        d_j=list_coo[j][1]
        distance=getDistance(d_i,d_j)
        id_stations=str(station1)+' '+str(station2)
        voc_distances[id_stations]=distance

In [124]:
# voc_distances['2 3']

In [131]:
#Applicazione “Delta Spaziale”
def giveSpatialWindow(line):
    lista=[]    
    time0=line[1][0]    #
    dic={}
    
    count_windows=len(line[1])#tot windows

    for station in time0:# only first window
        act_station=int(station.split('_')[0])
        #lista_station=[] 
        list_tmp=[]
        
        #for each window
        for i,window in enumerate(line[1]):           
            second_lista=[]
            #for each element of a window
            for all_el in window :
                #second_lista=[]
                
                act_all_el=int(all_el.split('_')[0])
                time_window= all_el.split('_')[1]
                state=all_el.split('_')[2]
               
                if act_station!=act_all_el:
                    
                    key=''
                    if act_station<act_all_el:
                        key=str(act_station)+' '+str(act_all_el)
                    else:
                        key=str(act_all_el)+' '+str(act_station)                    
                    
                    dist=voc_distances[key]
                    if dist<=maxDelta*th:
                        delta=0
                        for d in range(1,maxDelta+1):
                            if d*th>=dist:
                                delta=d
                                break                        
                        string=state+'_'+time_window+'_'+str(delta)
                        second_lista.append(string)
                else:
                    string=state+'_'+time_window+'_'+str(0)
                    second_lista.append(string)
                    
            if len(second_lista)>0:
                list_tmp.append(second_lista)
        lista.append(((line[0]+'|'+str(act_station)),list_tmp))
    
    return lista

In [132]:
spatial_app_list = []

for i, filtered_windows2 in enumerate(filtered_windows2_list):
    spatial_app=filtered_windows2.flatMap(giveSpatialWindow)
    print(f"num elements in slot{i+1}: {spatial_app.count()}")
    spatial_app_list.append(spatial_app)

# spatial_app=filtered_windows2.flatMap(giveSpatialWindow)
# print(spatial_app.count())

num elements in slot1: 18195
num elements in slot2: 13445
num elements in slot3: 13784
num elements in slot4: 11892
num elements in slot5: 9501
num elements in slot6: 11720


In [133]:
# spatial_app.collect()

# [('Window 1|59',
#   [['QuasiVuota_T0_0', 'QuasiVuota_T0_1', 'Vuota_T0_1', 'Vuota_T0_3', 'QuasiVuota_T0_3'],
#    ['QuasiVuota_T1_1', 'Vuota_T1_1', 'QuasiVuota_T1_3', 'QuasiVuota_T1_3', 'QuasiVuota_T1_3']]),
#  ('Window 1|67',
#   [['QuasiVuota_T0_1', 'QuasiVuota_T0_0', 'Vuota_T0_0', 'Vuota_T0_2', 'QuasiVuota_T0_2'],
#    ['QuasiVuota_T1_0', 'Vuota_T1_0', 'QuasiVuota_T1_2', 'QuasiVuota_T1_2', 'QuasiVuota_T1_3']]), ...]

In [134]:
def row_seq(line):
    true=line[1]
    string=Row(sequence=true)
    return string

In [135]:
spatial_list = []

for spatial_app in spatial_app_list:
    spatial=spatial_app.map(row_seq)
    spatial_list.append(spatial)

# spatial=spatial_app.map(row_seq)

In [136]:
# spatial.collect()

In [137]:
# seq_rdd =spatial.rdd.map(tuple)
filtered_seq_list = []

for spatial in spatial_list:
    filtered_seq = spatial.filter(lambda line: "T0" in line[0][0][0])
    filtered_seq_list.append(filtered_seq)


# filtered_seq = spatial.filter(lambda line: "T0" in line[0][0][0])
# filtered_seq.collect()

In [138]:
#create dataframes
df_list = []

for i, filtered_seq in enumerate(filtered_seq_list):
    df=filtered_seq.toDF()
    print (f"num elements in df{i+1}: {df.count()}")
    df_list.append(df)

# df=filtered_seq.toDF()
# len(df)

num elements in df1: 17453
num elements in df2: 12336
num elements in df3: 13377
num elements in df4: 10980
num elements in df5: 6965
num elements in df6: 9597


In [139]:
supports=[support]

In [140]:
print(len(df_list))
df_list[0].show(20,False)

6
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|sequence                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [141]:
#prefixspan to obtain sequence and frequence
prefix_list = []
len_prefix_list = []
for df in df_list:
    print(support)
    prefixSpan = PrefixSpan(minSupport=support, maxPatternLength=5,
                        maxLocalProjDBSize=5000)
    prefix=prefixSpan.findFrequentSequentialPatterns(df)   
    prefix_list.append(prefix)
    len_prefix=prefix.count()    
    prefix.show(len_prefix,False)
    len_prefix_list.append(len_prefix)


# counter=[]
# for support in supports:
#     print(support)
#     prefixSpan = PrefixSpan(minSupport=support, maxPatternLength=5,
#                         maxLocalProjDBSize=5000)
#     prefix=prefixSpan.findFrequentSequentialPatterns(df)   
#     len_prefix=prefix.count()    
# #     prefix.show(len_prefix,False)
#     counter.append(len_prefix)

0


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [142]:
len_prefix_list[0]

101579

In [143]:
pre_list = []

for prefix in prefix_list:
    pre=prefix.rdd.map(tuple)
    pre_list.append(pre)

# pre=prefix.rdd.map(tuple)

In [144]:
# pre.collect()

# [([['QuasiPiena_T0_1']], 22981),
#  ([['Piena_T1_1']], 9208),
#  ([['QuasiPiena_T2_1']], 21641),
#  ([['Piena_T2_2']], 12794),
#  ([['QuasiPiena_T2_3']], 18047),
#  ([['Piena_T2_3']], 6994),
#  ([['Piena_T1_0']], 17930),
#  ([['QuasiPiena_T0_3']], 18099), ... ]

In [145]:
def giveSelected(line):
    seq=line[0]
    found=False   
    for window in seq:
        for el in window:
            if 'T0' in el and '_0' in el:
                found=True
                break
    return found 

In [146]:
giveT0_list = []

for pre in pre_list:
    giveT0=pre.filter(giveSelected)
    giveT0_list.append(giveT0)

# giveT0=pre.filter(giveSelected)

In [147]:
# giveT0.collect()

#CONFIGURAZIONE 30min, 1000m, 0 supp, (3,3)


In [148]:
# df2=giveT0.toDF().withColumnRenamed('_1','sequence')
# df2=df2.withColumnRenamed('_2','freq')#.show(len_prefix,False)

In [149]:
#df2.show()

In [150]:
def mapValues(line):
    seq=line[0]
    final=''
    #voc[seq]=line[1]
    for i,window in enumerate(seq):
        if i>0:
            final+='-'
        for j,el in enumerate(window):
            if j>0:
                final+=','
            final+=el
    final+=(';'+str(line[1])+';'+str(i))
    return final  

In [151]:
mapDict_list = []

for giveT0 in giveT0_list:
    mapDict=giveT0.map(mapValues)
    mapDict_list.append(mapDict)


# mapDict=giveT0.map(mapValues)

In [152]:
# mapDict.collect()

# ['QuasiVuota_T0_0;282;0',
#  'Vuota_T0_0;192;0',
#  'QuasiVuota_T0_0,Vuota_T0_3;2;0',
#  'QuasiVuota_T0_0,Vuota_T0_3-QuasiVuota_T1_1;1;1',
#  'QuasiVuota_T0_0,Vuota_T0_3-QuasiVuota_T1_1,QuasiVuota_T1_3;1;1', ...]

In [153]:
li_list =[]

for mapDict in mapDict_list:
    li=mapDict.collect()
    li_list.append(li)

# li=mapDict.collect()
# voc={}

In [154]:
voc_list = []
for li in li_list:
    voc={}
    for el in li:
        splits=el.split(';')
        voc[splits[0]]=int(splits[1])
    voc_list.append(voc)


# for el in li:
#     splits=el.split(';')
#     voc[splits[0]]=int(splits[1])

In [155]:
# voc

# {'QuasiVuota_T0_0': 88948,
#  'Vuota_T0_0': 44987,
#  'Vuota_T0_0-QuasiVuota_T1_2': 33837,
#  'QuasiVuota_T0_0-Vuota_T1_0': 26011,
#  'QuasiVuota_T0_2,Vuota_T0_0': 34141, ...}

In [156]:
repeated_list = []

for voc in voc_list:
    repeated_el_window=0
    for el in voc.keys():
        flag_rep=False
        windows=el.split('-')
        for w in windows:
            tot_items=len(w.split(','))
            set_items=len(set(w.split(',')))
            if tot_items!=set_items:
                repeated_el_window+=1
                break
    repeated_list.append(repeated_el_window)
    
# repeated_list[0]


# repeated_el_window=0
# for el in voc.keys():
#     flag_rep=False
#     windows=el.split('-')
#     for w in windows:
#         tot_items=len(w.split(','))
#         set_items=len(set(w.split(',')))
#         if tot_items!=set_items:
#             repeated_el_window+=1
#             break
# repeated_el_window

In [157]:
voc_supports_list = []
keys_list = []
values_list = []

for voc in voc_list:
    voc_supports={}
    for el in voc.keys():    
        if len(el.split('-'))>1:        
            num=int(voc[el])       
            string=''
            tot=el.split('-')[:-1]
            for k,station in enumerate(tot):
                if k>0:
                    string+='-'
                string+=station
            #print(string)
            den=int(voc[string])
            voc_supports[el]=str(num/den)+' - '+str(voc[el])
    voc_supports_list.append(voc_supports)
#     keys=list(voc_supports.keys())
#     keys_list.append(keys)
#     values=list(voc_supports.values())
#     values_list.append(values)

In [158]:
# voc_supports

# {'Vuota_T0_0-QuasiVuota_T1_2': '0.7521506212905951 - 33837',
#  'QuasiVuota_T0_0-Vuota_T1_0': '0.29242928452579037 - 26011',
#  'Vuota_T0_0-QuasiVuota_T2_3': '0.4979438504456843 - 22401',
#  'Vuota_T0_0-Vuota_T2_3': '0.28939471402849715 - 13019',
#  'QuasiVuota_T0_0-QuasiVuota_T2_0': '0.3469780096236003 - 30863', ...}

In [159]:
#sort vocabulary by decreasing values and sort within each window
# tot_frequence=0
# for key in voc_supports:    
#     splitted=key.split('-')      
#     splitted.sort()
#     tmp_frequence=int(voc_supports[key].split(' - ')[1])
#     tot_frequence+=tmp_frequence
# for key in voc_supports:
#     freq= int(voc_supports[key].split(' - ')[1])
#     #print(freq)
# #     perc_value=float("{:.2f}".format(freq*100/tot_frequence))
#     voc_supports[key]=str(voc_supports[key].split(' - ')[0])+' - ' + str(freq)#+' - '+str(perc_value)+'%'  
# # voc_supports = dict(sorted(voc_supports.items(), key=operator.itemgetter(1),reverse=True))
# voc_supports = dict(sorted(voc_supports.items(), 
#                            key=lambda v: (float(v[1].split(' - ')[0]), int(v[1].split(' - ')[1])),
#                            reverse=True))

In [163]:
for i, voc_supports in enumerate(voc_supports_list):
    
    #sort vocabulary by decreasing values of confidence
    voc_supports = dict(sorted(voc_supports.items(), 
                               key=lambda v: (float(v[1].split(' - ')[0]), int(v[1].split(' - ')[1])),
                               reverse=True))
    
    #save file ordered by confidence
    file = open(eval("output_file_conf"+str(i+1)), "w")
    list_pattern=[]
    for el in voc_supports:
        key_list=[]
        for e in el.split('-'):
            key_list.append([e])
        list_pattern.append([[key_list], [voc_supports[el]]])
    file.write('TIME SLOT: ' + eval("output_file_conf"+str(i+1)).split('_')[1] + ' Pattern, Confidence-Frequence'+'\n')
    file.write(f'Total number of input patterns: {len(voc_supports)}'+'\n')
    for el in list_pattern:  
        file.write(str(el)+ '\n')    
    file.close()

In [ ]:
end = time.time()
print(f'The time of execution is: {end-start} s')